In [ ]:
from multiply_data_access import DataAccessComponent
import yaml
data_access_component = DataAccessComponent()
data_access_component.show_stores()
data_access_component.get_provided_data_types()
BARRAX_ROI = "POLYGON((-2.20397502663252 39.09868106889479,-1.9142106223355313 39.09868106889479," \
             "-1.9142106223355313 38.94504502508093,-2.20397502663252 38.94504502508093," \
             "-2.20397502663252 39.09868106889479))"
start_time = '2017-06-01'
end_time = '2017-06-10'
s2_data_infos = data_access_component.query(BARRAX_ROI, start_time, end_time, 'AWS_S2_L2')
print(s2_data_infos)
s2_urls = data_access_component.get_data_urls_from_data_set_meta_infos(s2_data_infos)
print(s2_urls)
import os
working_dir = '/home/user/m1/'
def create_dir(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
create_dir(working_dir)
s2_l2_dir = '{}/s2_l2'.format(working_dir)
from multiply_orchestration import create_sym_links
create_sym_links(s2_urls, s2_l2_dir)
configuration_file='{}/config.yaml'.format(os.getcwd())
with open(configuration_file) as config_file:
    parameters = yaml.load(config_file)
variables = 'n,cab,car,cbrown,cw,cm,lai,ala,bsoil,psoil'
priors_dir = '{}/priors'.format(working_dir)

In [ ]:
inference_engine_dir = '/software/inference-engine-0.4/multiply_inference_engine'
s2_emulators_dir = '/data/archive/emulators/s2_prosail'
inference_type = 'high'
state_mask = parameters['General']['state_mask']

biophys_dir = '{}/biophys'.format(working_dir)
create_dir(biophys_dir)
previous_state = '{}/state/2017-01-01'.format(working_dir)
create_dir(previous_state)
next_state = '{}/state/2017-01-10'.format(working_dir)
create_dir(next_state)

inference_engine_call = ( "python "+inference_engine_dir+"/inference_engine.py"
          + " -s " + str(start_time)
          + " -e " + str(end_time)
          + " -i " + inference_type
          + " -em " + s2_emulators_dir
          + " -p " + variables
          + ((" -ps " + previous_state) if previous_state != 'none' else "")
          + " -pd " + priors_dir
          + " -d " + s2_l2_dir
          + " -sm " + state_mask
          + " -o " + biophys_dir
          + " -ns " + next_state )
print(inference_engine_call)
os.system(inference_engine_call)